# Table of Contents
 <p><div class="lev1"><a href="#Task-1.-Compiling-Ebola-Data"><span class="toc-item-num">Task 1.&nbsp;&nbsp;</span>Compiling Ebola Data</a></div>
 <div class="lev1"><a href="#Task-2.-RNA-Sequences"><span class="toc-item-num">Task 2.&nbsp;&nbsp;</span>RNA Sequences</a></div>
 <div class="lev1"><a href="#Task-3.-Class-War-in-Titanic"><span class="toc-item-num">Task 3.&nbsp;&nbsp;</span>Class War in Titanic</a></div></p>

In [ ]:
import pandas as pd
import os
import numpy as np

## Task 1. Compiling Ebola Data

The `DATA_FOLDER/ebola` folder contains summarized reports of Ebola cases from three countries (Guinea, Liberia and Sierra Leone) during the recent outbreak of the disease in West Africa. For each country, there are daily reports that contain various information about the outbreak in several cities in each country.

Use pandas to import these data files into a single `Dataframe`.
**Using this `DataFrame`, calculate for *each country*, the *daily average* per year of *new cases* and *deaths*.
Make sure you handle all the different expressions for *new cases* and *deaths* that are used in the reports.**

## Task 2. RNA Sequences

In the `DATA_FOLDER/microbiome` subdirectory, there are 9 spreadsheets of microbiome data that was acquired from high-throughput RNA sequencing procedures, along with a 10<sup>th</sup> file that describes the content of each. 

Use pandas to import the first 9 spreadsheets into a single `DataFrame`.
Then, add the metadata information from the 10<sup>th</sup> spreadsheet as columns in the combined `DataFrame`.
Make sure that the final `DataFrame` has a unique index and all the `NaN` values have been replaced by the tag `unknown`.

In [ ]:
# Write your answer here

## Task 3. Class War in Titanic

Use pandas to import the data file `Data/titanic.xls`. It contains data on all the passengers that travelled on the Titanic.

In [ ]:
from IPython.core.display import HTML
HTML(filename='/home/thomas/Documents/ADA2017-Tutorials/02 - Intro to Pandas/Data/titanic.html')

For each of the following questions state clearly your assumptions and discuss your findings:
1. Describe the *type* and the *value range* of each attribute. Indicate and transform the attributes that can be `Categorical`. 
2. Plot histograms for the *travel class*, *embarkation port*, *sex* and *age* attributes. For the latter one, use *discrete decade intervals*. 
3. Calculate the proportion of passengers by *cabin floor*. Present your results in a *pie chart*.
4. For each *travel class*, calculate the proportion of the passengers that survived. Present your results in *pie charts*.
5. Calculate the proportion of the passengers that survived by *travel class* and *sex*. Present your results in *a single histogram*.
6. Create 2 equally populated *age categories* and calculate survival proportions by *age category*, *travel class* and *sex*. Present your results in a `DataFrame` with unique index.

### Question 1

We define the types of the attributes we will use in this task and then load the dataset in titanic.xls

In [ ]:
columns_types = {
    'pclass': np.uint8,
    'survived': np.bool_,
    'parch': np.uint8,
    'sibsp': np.uint8,
    'age': np.float32,
    'fare': np.float32,
}

titanic = pd.read_excel('/home/vinz/Desktop/ADA/ADA2017-Tutorials/02 - Intro to Pandas/Data/titanic.xls',
                        dtype=columns_types)

We transform the attributes pclass, sex and embarked into Categorical. It makes sense to do it with those attributes as the range of values is not very large and it will be useful in the future to display the bar plots.

We then rename the categories to make them easier to understand and visualize

In [ ]:
titanic.pclass = titanic.pclass.astype('category')
titanic.sex = titanic.sex.astype('category')
titanic.embarked = titanic.embarked.astype('category')

In [ ]:
titanic.pclass.cat.rename_categories(['First', 'Second', 'Third'], inplace=True)
titanic.sex.cat.rename_categories(['Female', 'Male'], inplace=True)
titanic.embarked.cat.rename_categories(['Cherbourg', 'Queenstown', 'Southampton'], inplace=True)

In [ ]:
titanic.head(n=10)

We display the type of each attribute and in the cell below the value range

In [ ]:
titanic.dtypes

In [ ]:
# Describe range of numeric variables
titanic[['age', 'sibsp', 'parch', 'fare']].describe().transpose()[['min', 'max']]

### Question 2

We first plot the histograms for the travel class, embarkation port and sex attributes

In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt

In [ ]:
titanic.pclass.value_counts().plot(kind='bar')
plt.title('travel class proportion')

In [ ]:
titanic.embarked.value_counts().plot(kind='bar')
plt.title('Embarkation port')

In [ ]:
titanic.sex.value_counts().plot(kind='bar')
plt.title('Sex proportion')

For this histogram we use discrete decade intervals from 0 to 80 (maximum age)

In [ ]:
pd.cut(titanic.age, [0, 10, 20, 30, 40, 50, 60, 70, 80], labels=[10, 20, 30, 40, 50, 60, 70, 80]).hist()
plt.title('Age distribution')

### Question 3

We are looking for the proportion of passengers by cabin floor. We thus copy the cabin attribute to drop the NAN without altering the original one. We copy/modify it in a way such that we just keep the letter of the cabin floor.

In [ ]:
floor_counts = titanic.cabin.copy()\
                            .str.extract('([A-Z])')\
                            .dropna()\
                            .value_counts()

We then present the results in a pie chart with the different letters of the cabin floors

In [ ]:
plt.pie(floor_counts, labels=floor_counts.index, autopct='%1.1f%%', startangle=140)
plt.axis('equal')
plt.show()

### Question 4

In this question we will present our results in three pie charts : one for each travel class representing the proportion of the passengers that survived.

So first of all we group the entries by pclass and compute for each pclass the mean of survived people. Which gives us the proportion if survived passengers

In [ ]:
survived = titanic[['pclass', 'survived']].groupby('pclass')\
                                          .mean()

In [ ]:
survived

We know plot a pie chart for each class

In [ ]:
labels = 'Survived', 'Not survived'

for class_ in titanic.pclass.cat.categories:
    ratios = survived['survived'][class_], 1 - survived['survived'][class_]
    plt.pie(ratios, labels=labels, autopct='%1.1f%%', startangle=140)
    plt.axis('equal')
    plt.title(' '.join([class_, 'class']))
    plt.show()

### Question 5

Here we want to represent the proportion of passengers that survived by travel class and sex.
To implement it we set the index (a multi index) of the titanic dataframe to (pclass, sex).
Thus we will be able to represent the "survival rate" in function of this index by grouping pclass and sex and applying the mean to the survived column

In [ ]:
survival_rates_class_sex = titanic.set_index(['pclass','sex'])\
                                  .groupby(level=('pclass','sex'),axis=0)[['survived']]\
                                  .mean()

In [ ]:
survival_rates_class_sex

We know represent our results in a single histogram

In [ ]:
survival_rates_class_sex.unstack().plot.bar()
plt.title('survival rate by travel class and sex')

### Question 6

In this question we will work on ages. We first decide to drop all the NaN in the column of ages. Then we will create the two categories of ages

In [ ]:
titanic2 = titanic.dropna(subset=['age']).copy()

To create the two age categories, we decide to use the median of all the ages which seems to be the best way of dividing the set. Hence we create the two categories "young" and "old" containing respectively the values [0, median] and [median+1, max]

In [ ]:
titanic_ages = pd.cut(titanic.age, [0, titanic2.age.median(), titanic2.age.max()], labels=['young','old'])
titanic2['age'] = titanic_ages

We now groupby the dataframe with pclass, sex and age as they are the attributes we are interested in and compute the mean of 'survived' values for those groups.

In [ ]:
result = titanic2.groupby(('pclass','sex','age'),axis=0)[['survived']]\
                 .mean()\
                 .round(3)\
                 .unstack('age')

In [ ]:
result

We now check that the index of our dataframe is unique

In [ ]:
result.index.is_unique